# Create environment

In [ ]:
!python -m venv myenv

In [ ]:
!myenv\Scripts\activate

# Installs and imports

In [1]:
#!pip install html2text wikitextparser tqdm pandas pdfminer.six pdf2image Pillow PyPDF2 pdfplumber

Download tesseract exe from here and install it to Program Files
https://digi.bib.uni-mannheim.de/tesseract/tesseract-ocr-w64-setup-5.3.3.20231005.exe

In [1]:
from tqdm import tqdm
from PIL import Image
from pathlib import Path
from threading import Thread
from urlextract import URLExtract
from html2text import html2text as htt
from pdf2image import convert_from_path
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTFigure, LTRect, LTChar
import wikitextparser as wtp
import pandas as pd
import pytesseract
import pdfplumber
import requests
import zipfile
import codecs
import PyPDF2
import json
import os
import re

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Set language and encoding for datasets

In [2]:
la = "sq" #change to your language
encoding='utf-8' #change depending on your language

# Prepare Folders

In [3]:
dir = "D:/alembedder" #change to your directory
downloads = Path(f'{dir}/downloads')
books = Path(f'{dir}/books')
data = Path(f'{dir}/data')
os.makedirs('downloads', exist_ok=True)
os.makedirs('data', exist_ok=True)
os.makedirs('books', exist_ok=True)
os.makedirs('kaggle', exist_ok=True)

# Helper Functions

In [4]:
extractor = URLExtract()

def remove_urls(text):
    urls = extractor.find_urls(text)
    for url in urls:
        text = text.replace(url, '')
    return text

def preview_large_file(file_path, lines=100, encoding=encoding):
    with open(file_path, 'r', encoding=encoding, errors='replace') as file:
        for _ in range(lines):
            line = file.readline()
            if not line:
                break
            print(line.strip())

# Extract text from Wikipedia XML dump

### Download XML dump

In [18]:
wiki_url = f'https://dumps.wikimedia.org/{la}wiki/latest/{la}wiki-latest-pages-articles.xml.bz2'
target_file = downloads / f'downloads/{la}wiki-latest-pages-articles.xml.bz2'
downloads.mkdir(parents=True, exist_ok=True)
with requests.get(wiki_url, stream=True) as r:
    r.raise_for_status()
    with open(target_file, 'wb') as file:
        for chunk in tqdm(r.iter_content(chunk_size=8192)): 
            file.write(chunk)
print(f"File downloaded to {target_file}")

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: ':\\alembedder\\dwonloads'

### Unzip dump

In [ ]:
with zipfile.ZipFile(target_file, 'r') as zip_ref:
    zip_ref.extractall(downloads)
print(f"Extracted files to {downloads}")

### Functions

In [3]:
def dewiki(text):
    text = wtp.parse(text).plain_text()  # wiki to plaintext 
    text = htt(text)  # remove any HTML
    text = text.replace('\\n',' ')  # replace newlines
    text = re.sub('\s+', ' ', text)  # replace excess whitespace
    return text

def analyze_chunk(text):
    try:
        if '<redirect title="' in text:  # this is not the main article
            return None
        if '(disambiguation)' in text:  # this is not an article
            return None
        else:
            title = text.split('<title>')[1].split('</title>')[0]
            title = htt(title)
            if ':' in title:  # most articles with : in them are not articles we care about
                return None
        serial = text.split('<id>')[1].split('</id>')[0]
        content = text.split('</text')[0].split('<text')[1].split('>', maxsplit=1)[1]
        content = dewiki(content)
        return {'title': title.strip(), 'text': content.strip(), 'id': serial.strip()}
    except Exception as oops:
        print(oops)
        return None

def old_save_article(article, savedir):
    doc = analyze_chunk(article)
    if doc:
        #print('SAVING:', doc['title']) #debuge
        filename = doc['id'] + '.json'
        with open(savedir + filename, 'w', encoding='utf-8') as outfile:
            json.dump(doc, outfile, sort_keys=True, indent=1, ensure_ascii=False)

def save_article(article, savedir):
    doc = analyze_chunk(article)
    if doc:
        # Use the ID as filename, but save as .txt
        filename = doc['id'] + '.txt'
        with open(savedir + filename, 'w', encoding='utf-8') as outfile:
            # Write only the content section to the file
            outfile.write(doc['text'])

def process_file_text(filename, savedir):
    article = ''
    with open(filename, 'r', encoding='utf-8') as infile:
        for line in tqdm(infile, desc="Processing files"):
            if '<page>' in line:
                article = ''
            elif '</page>' in line:  # end of article
                Thread(target=save_article, args=(article, savedir)).start()
            else:
                article += line

### Main Loop

In [4]:
xml_file = downloads / f'{la}wiki-latest-pages-articles-multistream.xml'

if __name__ == '__main__':
    process_file_text(xml_file, data)

Processing file: 9728611it [09:17, 17452.04it/s] 


# Kosovo News Articles Dataset - Kaggle 
Delete or replace with other dataset for other languages

### Kaggle API credentials stuff
Get a kaggle.json file by going to the settings of your Kaggle profile and putting it in the current directory

In [5]:
kaggle_dir = os.path.join('d:', os.sep, 'alembedder', '.kaggle')
os.rename(os.path.join('d:', os.sep, 'alembedder', 'kaggle.json'), 
          os.path.join(kaggle_dir, 'kaggle.json'))
os.environ['KAGGLE_CONFIG_DIR'] = kaggle_dir

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'd:\\alembedder\\kaggle.json' -> 'd:\\alembedder\\.kaggle\\kaggle.json'

### Download Kaggle Dataset

In [10]:
!kaggle datasets download -d gentrexha/kosovo-news-articles-dataset -p d:/alembedder #CHANGE PATH


  0%|          | 0.00/1.27G [00:00<?, ?B/s]
  0%|          | 1.00M/1.27G [00:00<15:17, 1.49MB/s]
  0%|          | 2.00M/1.27G [00:00<08:05, 2.81MB/s]
  0%|          | 4.00M/1.27G [00:00<04:03, 5.61MB/s]
  0%|          | 6.00M/1.27G [00:01<02:54, 7.79MB/s]
  1%|          | 8.00M/1.27G [00:01<02:58, 7.60MB/s]
  1%|          | 9.00M/1.27G [00:01<03:02, 7.43MB/s]
  1%|          | 10.0M/1.27G [00:01<02:57, 7.66MB/s]
  1%|          | 11.0M/1.27G [00:01<03:12, 7.05MB/s]
  1%|          | 12.0M/1.27G [00:02<03:09, 7.16MB/s]
  1%|          | 13.0M/1.27G [00:02<03:19, 6.78MB/s]
  1%|          | 14.0M/1.27G [00:02<04:42, 4.79MB/s]
  1%|          | 15.0M/1.27G [00:02<04:06, 5.48MB/s]
  1%|          | 16.0M/1.27G [00:02<03:42, 6.07MB/s]
  1%|▏         | 18.0M/1.27G [00:03<04:20, 5.19MB/s]
  1%|▏         | 19.0M/1.27G [00:03<03:48, 5.91MB/s]
  2%|▏         | 20.0M/1.27G [00:03<03:50, 5.85MB/s]
  2%|▏         | 21.0M/1.27G [00:03<03:44, 5.99MB/s]
  2%|▏         | 22.0M/1.27G [00:03<03:31, 6.37MB/s]
 

### Unzip dataset

In [15]:
zip_path = f'{dir}/data/kosovo-news-articles-dataset.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(downloads)
print(f"Extracted files to {downloads}")

Extracted files to D:/alembedder/data


### Turn dataset from CSV to TXT

In [35]:
kag = pd.read_csv('D:/alembedder/data/Kosovo-News-Articles.csv')

0    Kur Beatrice Borromeo u martua me Pierre Casir...
1    Gjatë kontrollit të automjetit zyrtarët polico...
2    Së fundmi Real Madrid dërgoi një ofertë 160 mi...
3    Enca Haxhia njihet si një ndër këngëtaret më s...
4    Gurët në veshka janë depozitime minerale që fo...
Name: content, dtype: object

In [43]:
file_path = 'D:/alembedder/data/kag.txt'

with open(file_path, 'w', encoding=encoding) as file:
    for index, row in kag.iterrows():
        title_str = str(row['title'])
        content_str = str(row['content'])
        file.write(title_str + '\n')
        file.write(content_str + '\n\n')

In [45]:
preview_large_file('data/kag_content.txt', 50) 

As Kate as Meghan; ja cila është princesha më me stil sipas prestigjozes “Tatler”
Kur Beatrice Borromeo u martua me Pierre Casiraghi e Monakos në 2015, ajo nuk veshi thjesht një fustan nusërie. Në vend të njërit, ajo zgjodhi katër fustane të ndryshme, dy nga Valentino për celebrimin e saj civil në Monako dhe dy nga Armani Privé për shërbimin fetar në Liqenin Maggiore. Nëse kjo nuk është dëshmi e stilit të saj të pagabueshëm, nuk jemi të sigurt se çfarë është tjetër! Mbretëresha 36-vjeçare evropiane është vajza e Don Carlo Ferdinando, aristokratit italian. Ajo punoi si gazetare politike, para se të martohej me Casiraghi, djali më të vogël të Caroline, Princeshës së Hanoverit dhe nipi i Grace Kelly. Ajo nuk është e pranishme në mediat për skandalet e saj por për elegancën që e karakterizon në çdo dalje publike. Nga eventet për Valentinon, Armanin Prive dhe Chanel, Beatreice e cila është dhe mama e dy fëmijëve shfaqet gjithnjë elegante dhe me shumë stil.

I kapen 10 kg substanca narkotike

### Remove URLs

In [6]:
kag_input_file_path = 'data/kag_content.txt'
kag_output_file_path = 'data/kag.txt'

with open(kag_input_file_path, 'r', encoding=encoding) as infile, open(kag_output_file_path, 'w', encoding=encoding) as outfile:
    for line in tqdm(infile):
        cleaned_line = remove_urls(line)
        outfile.write(cleaned_line)

0it [00:00, ?it/s]

961it [00:02, 460.25it/s]


KeyboardInterrupt: 

In [61]:
preview_large_file('data/kag.txt', 50) 

As Kate as Meghan; ja cila është princesha më me stil sipas prestigjozes “Tatler”
Kur Beatrice Borromeo u martua me Pierre Casiraghi e Monakos në 2015, ajo nuk veshi thjesht një fustan nusërie. Në vend të njërit, ajo zgjodhi katër fustane të ndryshme, dy nga Valentino për celebrimin e saj civil në Monako dhe dy nga Armani Privé për shërbimin fetar në Liqenin Maggiore. Nëse kjo nuk është dëshmi e stilit të saj të pagabueshëm, nuk jemi të sigurt se çfarë është tjetër! Mbretëresha 36-vjeçare evropiane është vajza e Don Carlo Ferdinando, aristokratit italian. Ajo punoi si gazetare politike, para se të martohej me Casiraghi, djali më të vogël të Caroline, Princeshës së Hanoverit dhe nipi i Grace Kelly. Ajo nuk është e pranishme në mediat për skandalet e saj por për elegancën që e karakterizon në çdo dalje publike. Nga eventet për Valentinon, Armanin Prive dhe Chanel, Beatreice e cila është dhe mama e dy fëmijëve shfaqet gjithnjë elegante dhe me shumë stil.

I kapen 10 kg substanca narkotike

# CC-100 Monolingual Dataset Download

In [17]:
file_url = f'https://data.statmt.org/cc-100/{la}.txt.xz'
target_file = downloads / f'{la}.txt'  
downloads.mkdir(parents=True, exist_ok=True)
with requests.get(file_url, stream=True) as r:
    r.raise_for_status()
    with open(target_file, 'wb') as file:
        for chunk in r.iter_content(chunk_size=8192): 
            file.write(chunk)
print(f"File downloaded to {target_file}")

File downloaded to d:\alembedder\data\sq.txt


### Preview dataset

In [34]:
preview_large_file(f'{la}.txt', 50) 

‘Lajm i kobshëm’, 'shpërthen' ish-presidenti pas mbylljes së shkollave: Është larguar trashëgimia e vendit
Pas lajmit për mbylljne e 16 shkollave në qarkun e Fierit ka reaguar ish presidenti Buajr Nishani, i cili e cilëson këtë si një lajm të kobshëm.
Nishani shkruan se këto tre vite të fundit kemi largimin më masiv që nga ai i 1990-1991 si shkak i largimit të familjeve shqiptare drejt emigracionit.
Eshtë larguar truri, energjia, familja, trashëgimia e vendit. Kjo është tradhëtia më e madhe kombëtare ndaj shqiptarëve që nga 1912-ta.
Ky fatalitet që po na kolapson në vend që të sfidohet, kthehet në komedi politike. Përgjegjësit e vetëm të përzënies sē fëmijve dhe mbylljes së shkollave çfaqen duke “dhuruar” libra.”-shkruan ndër të tjera Nishani.
Të dashur miq, librat e shkollës të fëmijëve shqiptar po mbeten jetim !
Pak ditë më parë një lajm pothuajse i kobshëm u bë prezent nga vetë autoritetet e qeverisë së Tiranës.
Në qarkun e Fierit mbyllen 16 shkolla !
Shkaku, largimi i familjeve shq

In [ ]:
cc_100_input_file_path = f'data/{la}.txt'
cc_100_output_file_path = f'data/{la}_clean.txt'

with open(cc_100_input_file_path, 'r', encoding=encoding) as infile, open(cc_100_output_file_path, 'w', encoding=encoding) as outfile:
    for line in tqdm(infile):
        cleaned_line = remove_urls(line)
        outfile.write(cleaned_line)

In [9]:
original_file = 'sq.txt'
utf8_file = 'utf8_sq.txt'

with codecs.open(original_file, 'r', encoding='utf-8', errors='ignore') as file:
    lines = file.read()

with codecs.open(utf8_file, 'w', encoding='utf-8') as file:
    file.write(lines)

preview_large_file(f'utf8_sq.txt', 50) 

In [11]:
preview_large_file('data/utf8_sq.txt', 50) 

‘Lajm i kobshëm’, 'shpërthen' ish-presidenti pas mbylljes së shkollave: Është larguar trashëgimia e vendit
Pas lajmit për mbylljne e 16 shkollave në qarkun e Fierit ka reaguar ish presidenti Buajr Nishani, i cili e cilëson këtë si një lajm të kobshëm.
Nishani shkruan se këto tre vite të fundit kemi largimin më masiv që nga ai i 1990-1991 si shkak i largimit të familjeve shqiptare drejt emigracionit.
Eshtë larguar truri, energjia, familja, trashëgimia e vendit. Kjo është tradhëtia më e madhe kombëtare ndaj shqiptarëve që nga 1912-ta.
Ky fatalitet që po na kolapson në vend që të sfidohet, kthehet në komedi politike. Përgjegjësit e vetëm të përzënies sē fëmijve dhe mbylljes së shkollave çfaqen duke “dhuruar” libra.”-shkruan ndër të tjera Nishani.
Të dashur miq, librat e shkollës të fëmijëve shqiptar po mbeten jetim !
Pak ditë më parë një lajm pothuajse i kobshëm u bë prezent nga vetë autoritetet e qeverisë së Tiranës.
Në qarkun e Fierit mbyllen 16 shkolla !
Shkaku, largimi i familjeve shq

# PDF books

### Helper Functions

In [5]:
def text_extraction(element):
    line_text = element.get_text()
    line_formats = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            for character in text_line:
                if isinstance(character, LTChar):
                    line_formats.append(character.fontname)
                    line_formats.append(character.size)
    format_per_line = list(set(line_formats))
    return (line_text, format_per_line)

def crop_image(element, pageObj):
    [image_left, image_top, image_right, image_bottom] = [element.x0,element.y0,element.x1,element.y1] 
    pageObj.mediabox.lower_left = (image_left, image_bottom)
    pageObj.mediabox.upper_right = (image_right, image_top)
    cropped_pdf_writer = PyPDF2.PdfWriter()
    cropped_pdf_writer.add_page(pageObj)
    with open('cropped_image.pdf', 'wb') as cropped_pdf_file:
        cropped_pdf_writer.write(cropped_pdf_file)

def convert_to_images(input_file,):
    images = convert_from_path(input_file)
    image = images[0]
    output_file = "PDF_image.png"
    image.save(output_file, "PNG")

def image_to_text(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text

def extract_table(pdf_path, page_num, table_num):
    pdf = pdfplumber.open(pdf_path)
    table_page = pdf.pages[page_num]
    table = table_page.extract_tables()[table_num]
    return table

def table_converter(table):
    table_string = ''
    for row_num in range(len(table)):
        row = table[row_num]
        cleaned_row = [item.replace('\n', ' ') if item is not None and '\n' in item else 'None' if item is None else item for item in row]
        table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')
    table_string = table_string[:-1]
    return table_string

### Main function

In [18]:
def process_pdf(pdf_path):
    pdfFileObj = open(pdf_path, 'rb')
    pdfReaded = PyPDF2.PdfReader(pdfFileObj)
    text_per_page = {}
    for pagenum, page in enumerate(extract_pages(pdf_path)):
        pageObj = pdfReaded.pages[pagenum]
        page_text = []
        line_format = []
        text_from_images = []
        text_from_tables = []
        page_content = []
        table_num = 0
        first_element = True
        table_extraction_flag = False
        pdf = pdfplumber.open(pdf_path)
        page_tables = pdf.pages[pagenum]
        tables = page_tables.find_tables()
        page_elements = [(element.y1, element) for element in page._objs]
        page_elements.sort(key=lambda a: a[0], reverse=True)
        lower_side = None
        upper_side = None
        for i, component in enumerate(page_elements):
            pos = component[0]
            element = component[1]
            if isinstance(element, LTTextContainer):
                if not table_extraction_flag:
                    (line_text, format_per_line) = text_extraction(element)
                    page_text.append(line_text)
                    line_format.append(format_per_line)
                    page_content.append(line_text)
            if isinstance(element, LTFigure):
                crop_image(element, pageObj)
                convert_to_images('cropped_image.pdf')
                image_text = image_to_text('PDF_image.png')
                text_from_images.append(image_text)
                page_content.append(image_text)
                page_text.append('image')
                line_format.append('image')
            if isinstance(element, LTRect):
                if first_element and (table_num + 1) <= len(tables):
                    lower_side = page.bbox[3] - tables[table_num].bbox[3]
                    upper_side = element.y1 
                    table = extract_table(pdf_path, pagenum, table_num)
                    table_string = table_converter(table)
                    text_from_tables.append(table_string)
                    page_content.append(table_string)
                    table_extraction_flag = True
                    first_element = False
                    page_text.append('table')
                    line_format.append('table')
                elif lower_side is not None and upper_side is not None:
                    if element.y0 >= lower_side and element.y1 <= upper_side:
                        pass
                    elif i + 1 < len(page_elements) and not isinstance(page_elements[i+1][1], LTRect):
                        table_extraction_flag = False
                        first_element = True
                        table_num += 1
        dctkey = 'Page_' + str(pagenum)
        text_per_page[dctkey] = [page_text, line_format, text_from_images, text_from_tables, page_content]
    pdfFileObj.close()
    if os.path.exists('cropped_image.pdf'):
        os.remove('cropped_image.pdf')
    if os.path.exists('PDF_image.png'):
        os.remove('PDF_image.png')
    return '\n'.join([''.join(page_content) for page_content in [page_data[4] for page_data in text_per_page.values()]])

### Run

In [19]:
directory = 'D:/alembedder/books'
output_file = 'books.txt'

with open(output_file, 'w', encoding=encoding) as text_file:
    for filename in tqdm(os.listdir(directory)):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(directory, filename)
            extracted_text = process_pdf(pdf_path)
            #text_file.write(f'--- Text from {filename} ---\n')
            text_file.write(extracted_text)
            text_file.write('\n\n')

The PDF <_io.BufferedReader name='D:/alembedder/books\\endrrat_e_thyera.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
Illegal character in Name Object (b'/D\xdcP RENG\xdc')
The PDF <_io.BufferedReader name='D:/alembedder/books\\hstrmili_sikur_te_isha_djal.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
Multiple definitions in dictionary at byte 0x301c5 for key /Type
The PDF <_io.BufferedReader name='D:/alembedder/books\\tahir_meha.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='D:/alembedder/books\\Vrasesi_i_qymyrit.pdf'> contains a